In [2]:
# Dependencies
import pandas as pd
import numpy as np

In [3]:
# Initialize variables
cutoff_year = 2000

test_data_cutoff_year = 2022
prediction_data_cutoff_year = test_data_cutoff_year + 1

# Data Cleanup

In [5]:
# Read in CSV file 
team_summaries = pd.read_csv(r"C:\Users\jbwil\Downloads\Project\Resources\Team_Summaries.csv")
team_summaries = team_summaries[['season','abbreviation','playoffs','w','l']]
team_summaries.rename(columns={'abbreviation':'tm'}, inplace=True)
team_summaries['playoffs'] = team_summaries['playoffs'].astype(int)
# team_summaries

In [6]:
# Read in CSV file
all_nba_df = pd.read_csv(r"C:\Users\jbwil\Downloads\Project\Resources\End_of_Season_Teams.csv")
# all_nba_df.head()

In [7]:
# Filter All-NBA players after year 2010. Keep only columns: all_nba, seas_id
modern_all_nba = all_nba_df[(all_nba_df['season'] >= cutoff_year) & (all_nba_df['type'] == 'All-NBA')][['type','seas_id']]

print(modern_all_nba.info())
modern_all_nba.head()

<class 'pandas.core.frame.DataFrame'>
Index: 347 entries, 14 to 810
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   type     347 non-null    object 
 1   seas_id  347 non-null    float64
dtypes: float64(1), object(1)
memory usage: 8.1+ KB
None


,type,seas_id
14,All-NBA,67.0
15,All-NBA,13.0
16,All-NBA,591.0
17,All-NBA,30158.0
18,All-NBA,30247.0


In [8]:
# Read in CSV file
player_totals_df = pd.read_csv(r"C:\Users\jbwil\Downloads\Project\Resources\combined_file.csv")

# Filter data after cutoff_year
player_totals_df =  player_totals_df[player_totals_df['season'] >= cutoff_year]

# player_totals_df.head()

In [9]:
# Find traded players current team
traded_players = player_totals_df.sort_values(by='seas_id', ascending=False).drop_duplicates(['player','season'])[['seas_id','season','player','tm']]

player_totals = pd.merge(player_totals_df, traded_players, on=['player','season'], how='left')

player_totals.rename(columns={'seas_id_x':'seas_id','tm_y':'tm'}, inplace=True)
player_totals.drop('seas_id_y', axis=1, inplace=True)

player_totals = pd.merge(player_totals, team_summaries, on=['season','tm'], how='left')

player_totals = player_totals.sort_values(by='seas_id').drop_duplicates(['season','player'])

print(player_totals.head().to_markdown())

|    |   seas_id |   season |   player_id | player           |   birth_year | pos   |   age | lg   | tm_x   |   g |   gs |   mp |   fg |   fga |   fg_percent |   x3p |   x3pa |   x3p_percent |   x2p |   x2pa |   x2p_percent |   e_fg_percent |   ft |   fta |   ft_percent |   orb |   drb |   trb |   ast |   stl |   blk |   tov |   pf |   pts | tm   |   playoffs |   w |   l |
|---:|----------:|---------:|------------:|:-----------------|-------------:|:------|------:|:-----|:-------|----:|-----:|-----:|-----:|------:|-------------:|------:|-------:|--------------:|------:|-------:|--------------:|---------------:|-----:|------:|-------------:|------:|------:|------:|------:|------:|------:|------:|-----:|------:|:-----|-----------:|----:|----:|
|  0 |         1 |     2022 |           1 | Precious Achiuwa |          nan | C     |    23 | NBA  | TOR    |  55 |   12 | 1140 |  196 |   404 |        0.485 |    29 |    108 |         0.269 |   167 |    296 |         0.564 |          0.521 |   87 

In [10]:
# Filter data after 2010
modern_player_totals = player_totals.copy()

# Categorize positions into 3 categories
modern_player_totals['forward'] = modern_player_totals['pos'].str.contains('f', case=False).astype(int)
modern_player_totals['guard'] = modern_player_totals['pos'].str.contains('g', case=False).astype(int)
modern_player_totals['center'] = modern_player_totals['pos'].str.contains('c', case=False).astype(int)

# Drop unnecessary columns
modern_player_totals.drop(['birth_year','player_id','player','lg','tm_x','pos','fg_percent','x3p_percent','x2p_percent','ft_percent','e_fg_percent','tm'], axis=1, inplace=True)


#print(modern_player_totals.info())
print(modern_player_totals.head().to_markdown())

|    |   seas_id |   season |   age |   g |   gs |   mp |   fg |   fga |   x3p |   x3pa |   x2p |   x2pa |   ft |   fta |   orb |   drb |   trb |   ast |   stl |   blk |   tov |   pf |   pts |   playoffs |   w |   l |   forward |   guard |   center |
|---:|----------:|---------:|------:|----:|-----:|-----:|-----:|------:|------:|-------:|------:|-------:|-----:|------:|------:|------:|------:|------:|------:|------:|------:|-----:|------:|-----------:|----:|----:|----------:|--------:|---------:|
|  0 |         1 |     2022 |    23 |  55 |   12 | 1140 |  196 |   404 |    29 |    108 |   167 |    296 |   87 |   124 |   100 |   228 |   328 |    50 |    31 |    30 |    59 |  102 |   508 |          1 |  48 |  34 |         0 |       0 |        1 |
|  1 |         2 |     2022 |    29 |  42 |   42 | 1133 |  157 |   263 |     0 |      1 |   157 |    262 |   47 |   129 |   214 |   271 |   485 |    97 |    36 |    46 |    79 |   98 |   361 |          1 |  56 |  26 |         0 |       0 |        

In [11]:
# Merge ALL NBA members with players
merged_nba = pd.merge(modern_all_nba, modern_player_totals, on='seas_id', how='right')
merged_nba['all_nba'] = merged_nba['type'].notnull().astype(int)

merged_nba.drop('type', axis=1, inplace=True)

# print(merged_nba.info())
# print(merged_nba.head(40).to_markdown())
# merged_nba.to_csv("Resources/merged_test.csv")

# Machine Learning

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [13]:
test_data = merged_nba[merged_nba['season'] <= test_data_cutoff_year]

# Split the data into X_train, X_test, y_train, y_test
X = test_data.drop("all_nba", axis=1)
y = test_data["all_nba"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Scale the features using a standard scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# By positions
Xg = X[X['guard'] == 1].drop(['forward','center'], axis=1)
yg = test_data[test_data['guard'] == 1]['all_nba']
Xg_train, Xg_test, yg_train, yg_test = train_test_split(Xg, yg, random_state=1)

Xf = X[X['forward'] == 1].drop(['guard','center'], axis=1)
yf = test_data[test_data['forward'] == 1]['all_nba']
Xf_train, Xf_test, yf_train, yf_test = train_test_split(Xf, yf, random_state=1)

Xc = X[X['center'] == 1].drop(['guard','forward'], axis=1)
yc = test_data[test_data['center'] == 1]['all_nba']
Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, random_state=1)

scalerg = StandardScaler()
Xg_train_scaled = scalerg.fit_transform(Xg_train)
Xg_test_scaled = scalerg.transform(Xg_test)

scalerf = StandardScaler()
Xf_train_scaled = scalerf.fit_transform(Xf_train)
Xf_test_scaled = scalerf.transform(Xf_test)

scalerc = StandardScaler()
Xc_train_scaled = scalerc.fit_transform(Xc_train)
Xc_test_scaled = scalerc.transform(Xc_test)

# Logistic Regression Model

In [14]:
# Train a Logistic Regression model and print the model score
classifier = LogisticRegression(max_iter=10000)

classifier.fit(X_train_scaled, y_train)

print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")
print('-------------------------------------')

# By positions
classifierg = LogisticRegression(max_iter=10000)
classifierg.fit(Xg_train_scaled, yg_train)

print(f"Training Data Score: {classifierg.score(Xg_train_scaled, yg_train)}")
print(f"Testing Data Score: {classifierg.score(Xg_test_scaled, yg_test)}")
print('-------------------------------------')

classifierf = LogisticRegression(max_iter=10000)
classifierf.fit(Xf_train_scaled, yf_train)

print(f"Training Data Score: {classifierf.score(Xf_train_scaled, yf_train)}")
print(f"Testing Data Score: {classifierf.score(Xf_test_scaled, yf_test)}")
print('-------------------------------------')

classifierc = LogisticRegression(max_iter=10000)
classifierc.fit(Xc_train_scaled, yc_train)

print(f"Training Data Score: {classifierc.score(Xc_train_scaled, yc_train)}")
print(f"Testing Data Score: {classifierc.score(Xc_test_scaled, yc_test)}")

Training Data Score: 0.9975247524752475
Testing Data Score: 0.9629629629629629
-------------------------------------
Training Data Score: 1.0
Testing Data Score: 0.9821428571428571
-------------------------------------
Training Data Score: 1.0
Testing Data Score: 0.9433962264150944
-------------------------------------
Training Data Score: 1.0
Testing Data Score: 1.0


In [15]:
# Make predictions on new data
prediction_data = merged_nba[merged_nba['season'] == prediction_data_cutoff_year].drop('all_nba', axis=1)
prediction_data_scaled = scaler.transform(prediction_data)
proba = classifier.predict_proba(prediction_data_scaled)

# Add the All-NBA probability to the prediction_data dataframe as percentage
prediction_data['All-NBA Probability'] = np.around(proba[:, 1] * 100 , 2)
prediction_data.sort_values(by='All-NBA Probability', ascending=False, inplace=True)

# By positions
prediction_gdata = merged_nba[(merged_nba['season'] == prediction_data_cutoff_year) & (merged_nba['guard']==1)].drop(['all_nba','forward','center'], axis=1)
prediction_gdata_scaled = scalerg.transform(prediction_gdata)
proba_g = classifierg.predict_proba(prediction_gdata_scaled)

# Add the All-NBA probability to the prediction_data dataframe as percentage
prediction_gdata['All-NBA G Probability'] = np.around(proba_g[:, 1] * 100 , 2)
prediction_gdata.sort_values(by='All-NBA G Probability', ascending=False, inplace=True)

prediction_fdata = merged_nba[(merged_nba['season'] == prediction_data_cutoff_year) & (merged_nba['forward']==1)].drop(['all_nba','guard','center'], axis=1)
prediction_fdata_scaled = scalerf.transform(prediction_fdata)
proba_f = classifierf.predict_proba(prediction_fdata_scaled)

# Add the All-NBA probability to the prediction_data dataframe as percentage
prediction_fdata['All-NBA F Probability'] = np.around(proba_f[:, 1] * 100 , 2)
prediction_fdata.sort_values(by='All-NBA F Probability', ascending=False, inplace=True)

prediction_cdata = merged_nba[(merged_nba['season'] == prediction_data_cutoff_year) & (merged_nba['center']==1)].drop(['all_nba','guard','forward'], axis=1)
prediction_cdata_scaled = scalerc.transform(prediction_cdata)
proba_c = classifierc.predict_proba(prediction_cdata_scaled)

# Add the All-NBA probability to the prediction_data dataframe as percentage
prediction_cdata['All-NBA C Probability'] = np.around(proba_c[:, 1] * 100 , 2)
prediction_cdata.sort_values(by='All-NBA C Probability', ascending=False, inplace=True)

In [16]:
# Merge in player names with correct Team name.  Correcting erroneous players with TOT as team name
player_info = player_totals.sort_values(by='seas_id', ascending=False)[['seas_id','player','tm']]

predicted_players = pd.merge(player_info, prediction_data, on='seas_id', how='right')

predicted_g_players = pd.merge(player_info, prediction_gdata, on='seas_id', how='right')
predicted_f_players = pd.merge(player_info, prediction_fdata, on='seas_id', how='right')
predicted_c_players = pd.merge(player_info, prediction_cdata, on='seas_id', how='right')

In [17]:
# Separate players by positions
top_guards = predicted_players[predicted_players['guard'] == 1].head(20)
top_forwards = predicted_players[predicted_players['forward'] == 1].head(10)
top_centers = predicted_players[predicted_players['center'] == 1].head(10)

# By positions
top_gguards = predicted_g_players[predicted_g_players['guard'] == 1].head(20)
top_fforwards = predicted_f_players[predicted_f_players['forward'] == 1].head(10)
top_ccenters = predicted_c_players[predicted_c_players['center'] == 1].head(10)


# Random Forest Model

In [18]:
# Train a Random Forest model and print the model score
rf_classifier = RandomForestClassifier(random_state=1)
rf_classifier.fit(X_train_scaled, y_train)

print(f"Random Forest Training Data Score: {rf_classifier.score(X_train_scaled, y_train)}")
print(f"Random Forest Testing Data Score: {rf_classifier.score(X_test_scaled, y_test)}")
print('-------------------------------------')

# By positions
rf_classifierg = RandomForestClassifier(random_state=1)
rf_classifierg.fit(Xg_train_scaled, yg_train)

print(f"Random Forest Training Data Score: {rf_classifierg.score(Xg_train_scaled, yg_train)}")
print(f"Random Forest Testing Data Score: {rf_classifierg.score(Xg_test_scaled, yg_test)}")
print('-------------------------------------')

rf_classifierf = RandomForestClassifier(random_state=1)
rf_classifierf.fit(Xf_train_scaled, yf_train)

print(f"Random Forest Training Data Score: {rf_classifierf.score(Xf_train_scaled, yf_train)}")
print(f"Random Forest Testing Data Score: {rf_classifierf.score(Xf_test_scaled, yf_test)}")
print('-------------------------------------')

rf_classifierc = RandomForestClassifier(random_state=1)
rf_classifierc.fit(Xc_train_scaled, yc_train)

print(f"Random Forest Training Data Score: {rf_classifierc.score(Xc_train_scaled, yc_train)}")
print(f"Random Forest Testing Data Score: {rf_classifierc.score(Xc_test_scaled, yc_test)}")


Random Forest Training Data Score: 1.0
Random Forest Testing Data Score: 0.9555555555555556
-------------------------------------
Random Forest Training Data Score: 1.0
Random Forest Testing Data Score: 0.9821428571428571
-------------------------------------
Random Forest Training Data Score: 1.0
Random Forest Testing Data Score: 0.9433962264150944
-------------------------------------
Random Forest Training Data Score: 1.0
Random Forest Testing Data Score: 1.0


In [19]:
# Make predictions on new data using Random Forest
rf_proba = rf_classifier.predict_proba(prediction_data_scaled)
prediction_data['All-NBA RF Probability'] = np.around(rf_proba[:, 1] * 100, 2)

# By positions
rf_proba_g = rf_classifierg.predict_proba(prediction_gdata_scaled)
prediction_gdata['All-NBA G RF Probability'] = np.around(rf_proba_g[:, 1] * 100, 2)

rf_proba_f = rf_classifierf.predict_proba(prediction_fdata_scaled)
prediction_fdata['All-NBA F RF Probability'] = np.around(rf_proba_f[:, 1] * 100, 2)

rf_proba_c = rf_classifierc.predict_proba(prediction_cdata_scaled)
prediction_cdata['All-NBA C RF Probability'] = np.around(rf_proba_c[:, 1] * 100, 2)

In [20]:
# Merge in player names with correct Team name for Random Forest probabilities
rf_predicted_players = pd.merge(player_info, prediction_data, on='seas_id', how='right')
rf_predicted_g_players = pd.merge(player_info, prediction_gdata, on='seas_id', how='right')
rf_predicted_f_players = pd.merge(player_info, prediction_fdata, on='seas_id', how='right')
rf_predicted_c_players = pd.merge(player_info, prediction_cdata, on='seas_id', how='right')

In [21]:
# Separate players by positions based on Random Forest probabilities
rf_top_guards = rf_predicted_players[rf_predicted_players['guard'] == 1].sort_values(by='All-NBA RF Probability', ascending=False).head(20)
rf_top_forwards = rf_predicted_players[rf_predicted_players['forward'] == 1].sort_values(by='All-NBA RF Probability', ascending=False).head(10)
rf_top_centers = rf_predicted_players[rf_predicted_players['center'] == 1].sort_values(by='All-NBA RF Probability', ascending=False).head(10)

# By positions based on Random Forest probabilities
rf_top_gguards = rf_predicted_g_players[rf_predicted_g_players['guard'] == 1].sort_values(by='All-NBA G RF Probability', ascending=False).head(20)
rf_top_fforwards = rf_predicted_f_players[rf_predicted_f_players['forward'] == 1].sort_values(by='All-NBA F RF Probability', ascending=False).head(10)
rf_top_ccenters = rf_predicted_c_players[rf_predicted_c_players['center'] == 1].sort_values(by='All-NBA C RF Probability', ascending=False).head(10)


# Logistic Regression Output

In [25]:
# Assuming the column for NBA probabilities is named 'NBA_Probability'
# Adjust the column name if it's different in your data frames

# Sort the data frames
top_guards = top_gguards.sort_values(by='All-NBA G Probability', ascending=False)
top_forwards = top_fforwards.sort_values(by='All-NBA F Probability', ascending=False)
top_centers = top_ccenters.sort_values(by='All-NBA C Probability', ascending=False)

# Print the top entries in each sorted data frame
print("Top Guards")
print(top_gguards.head(10).to_markdown())
print('-----------------------------------------------------------------------------------------------------')

print("Top Forwards")
print(top_fforwards.head(10).to_markdown())
print('-----------------------------------------------------------------------------------------------------')

print("Top Centers")
print(top_ccenters.head(6).to_markdown())


Top Guards
|    |   seas_id | player                   | tm   |   season |   age |   g |   gs |   mp |   fg |   fga |   x3p |   x3pa |   x2p |   x2pa |   ft |   fta |   orb |   drb |   trb |   ast |   stl |   blk |   tov |   pf |   pts |   playoffs |   w |   l |   guard |   All-NBA G Probability |
|---:|----------:|:-------------------------|:-----|---------:|------:|----:|-----:|-----:|-----:|------:|------:|-------:|------:|-------:|-----:|------:|------:|------:|------:|------:|------:|------:|------:|-----:|------:|-----------:|----:|----:|--------:|------------------------:|
|  0 |       938 | Damian Lillard           | MIL  |     2023 |    33 |  18 |   18 |  630 |  132 |   308 |    51 |    152 |    81 |    156 |  144 |   156 |    13 |    69 |    82 |   125 |    19 |     2 |    50 |   25 |   459 |          0 |  44 |  17 |       1 |                    3.25 |
|  1 |       895 | Reggie Jackson           | DEN  |     2023 |    33 |  21 |   13 |  565 |  116 |   228 |    35 |     86 |  

# Random Forest Output

In [ ]:
print("Top Guards")
print(rf_top_guards.head(10).to_markdown())
print('-----------------------------------------------------------------------------------------------------')
print("Top Forwards")
print(rf_top_forwards.head(10).to_markdown())
print('-----------------------------------------------------------------------------------------------------')
print("Top Centers")
print(rf_top_centers.head(6).to_markdown())

Top Guards
|     |   seas_id | player              | tm   |   season |   age |   g |   gs |   mp |   fg |   fga |   x3p |   x3pa |   x2p |   x2pa |   ft |   fta |   orb |   drb |   trb |   ast |   stl |   blk |   tov |   pf |   pts |   playoffs |   w |   l |   forward |   guard |   center |   All-NBA Probability |   All-NBA RF Probability |
|----:|----------:|:--------------------|:-----|---------:|------:|----:|-----:|-----:|-----:|------:|------:|-------:|------:|-------:|-----:|------:|------:|------:|------:|------:|------:|------:|------:|-----:|------:|-----------:|----:|----:|----------:|--------:|---------:|----------------------:|-------------------------:|
| 148 |      1155 | Russell Westbrook   | LAC  |     2023 |    35 |  19 |   10 |  507 |   90 |   207 |    14 |     51 |    76 |    156 |   34 |    54 |    29 |    97 |   126 |   100 |    25 |     6 |    44 |   46 |   228 |          0 |  33 |  31 |         0 |       1 |        0 |                  0.04 |                     

In [26]:
print("Top Guards")
print(rf_top_gguards.head(10).to_markdown())
print('-----------------------------------------------------------------------------------------------------')
print("Top Forwards")
print(rf_top_fforwards.head(10).to_markdown())
print('-----------------------------------------------------------------------------------------------------')
print("Top Centers")
print(rf_top_ccenters.head(6).to_markdown())

Top Guards
|     |   seas_id | player             | tm   |   season |   age |   g |   gs |   mp |   fg |   fga |   x3p |   x3pa |   x2p |   x2pa |   ft |   fta |   orb |   drb |   trb |   ast |   stl |   blk |   tov |   pf |   pts |   playoffs |   w |   l |   guard |   All-NBA G Probability |   All-NBA G RF Probability |
|----:|----------:|:-------------------|:-----|---------:|------:|----:|-----:|-----:|-----:|------:|------:|-------:|------:|-------:|-----:|------:|------:|------:|------:|------:|------:|------:|------:|-----:|------:|-----------:|----:|----:|--------:|------------------------:|---------------------------:|
| 116 |       786 | Dexter Dennis      | DAL  |     2023 |    24 |   2 |    0 |    7 |    2 |     6 |     0 |      3 |     2 |      3 |    0 |     1 |     1 |     1 |     2 |     0 |     0 |     0 |     0 |    0 |     4 |          0 |  32 |  31 |       1 |                    0.03 |                         16 |
| 178 |      1154 | Blake Wesley       | SAS  |     2